# Exploratory Data Analysis (EDA) Image Analysis 

## Emotion Face Classifier Notebook 3

Focuses on showing example expressions and aggregrate representations of emotion categories.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datascifuncs.tidbit_tools import load_json, write_json, print_json, check_directory_name

In [ ]:
main_dir = 'EmotionFaceClassifier'
check_directory_name(main_dir)

In [ ]:
from utils.image_processing import (
    display_random_images_from_df,
    preprocess_image,
    apply_pca,
    compute_average_face,
    log_pca_results
)

In [ ]:
# Read in FER 2013 data
fer2013_path = 'data/fer2013_paths.csv'
fer2013 = pd.read_csv(fer2013_path)

In [ ]:
fer2013.head()

In [ ]:
# Load common dicts from json config file
common_dicts = load_json('./configs/input_mappings.json')
print_json(common_dicts)

In [ ]:
color_dict = common_dicts['plotly_styles']['Training']['color']

## Display Example Images from Each Category

In [ ]:
display_random_images_from_df(fer2013, n_rows=3)

## Unsupervised Learning Analyses

In [ ]:
# Example: datasets/emotions/happy, datasets/emotions/sad, etc.
emotions = fer2013['emotion'].tolist()

In [ ]:
# Dictionary to hold images by category
images_by_category = {emotion: [] for emotion in emotions}

In [ ]:
# Group images by emotion
emotion_groups = fer2013.groupby('emotion')

# Iterate over each emotion category efficiently
for emotion, group in emotion_groups:
    # Preprocess each image using a list comprehension
    images = [preprocess_image(cv2.imread(img_path)) for img_path in group['img_path']]
    
    # Store preprocessed images for this emotion
    images_by_category[emotion] = images

In [ ]:
# Apply PCA and compute the average face for each category
n_components = 50
for emotion, images in images_by_category.items():
    print(f"Processing category: {emotion}")
    
    # Apply PCA
    transformed_images, pca = apply_pca(images, n_components)
    
    # Compute the average face
    average_face = compute_average_face(images)
    
    # Log the PCA and average face results with MLflow
    log_pca_results(pca, average_face, emotion)
    
    # Display the average face
    plt.imshow(average_face, cmap='gray')
    plt.title(f"Average Face: {emotion}")
    plt.show()

In [ ]:
# !mlflow ui

In [ ]:
import os
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
import cv2

# # Define emotion-to-color mapping
# emotion_colors = {
#     'happy': 'yellow',
#     'sad': 'blue',
#     'angry': 'red',
#     'surprised': 'green'
# }

# Directories where images and PCA arrays are stored
average_face_dir = 'imgs/facial_features'
pca_array_dir = 'data/pca_arrays'

# Emotions you're working with
emotions = common_dicts['emo_dict'].values()

# Create a figure with subplots
fig, axes = plt.subplots(2, len(emotions), figsize=(15, 10))  # 2 rows for PCA and average faces

# Iterate over emotions and create subplots for PCA and average faces
for idx, emotion in enumerate(emotions):
    
    # Load the average face image
    avg_face_path = os.path.join(average_face_dir, f"average_face_{emotion}.png")
    avg_face_img = cv2.imread(avg_face_path, cv2.IMREAD_GRAYSCALE)
    
    # Load the PCA components or a reconstructed image from PCA if you have one
    pca_components_path = os.path.join(pca_array_dir, f"pca_components_{emotion}.npy")
    pca_components = np.load(pca_components_path)  # Loaded PCA array (adjust how you visualize this)
    
    # ------------------------ Plotting Average Face ------------------------
    ax_avg = axes[0, idx]
    ax_avg.imshow(avg_face_img, cmap='gray')
    ax_avg.set_title(f'Average Face: {emotion}')
    ax_avg.axis('off')  # Hide axis for clean display
    
    # Add a colored border frame around the average face plot
    rect_avg = Rectangle((0, 0), 1, 1, transform=ax_avg.transAxes,
                         linewidth=5, edgecolor=color_dict[emotion], facecolor='none')
    ax_avg.add_patch(rect_avg)

    # ------------------------ Plotting PCA Image ------------------------
    ax_pca = axes[1, idx]
    
    # Assuming you have a way to visualize PCA components or a reconstructed face
    # For illustration, let's plot the first principal component reshaped to image size
    pca_img = pca_components[0].reshape(avg_face_img.shape)  # Reshape based on your image dimensions
    ax_pca.imshow(pca_img, cmap='gray')
    ax_pca.set_title(f'PCA Face: {emotion}')
    ax_pca.axis('off')

    # Add a colored border frame around the PCA face plot
    rect_pca = Rectangle((0, 0), 1, 1, transform=ax_pca.transAxes,
                         linewidth=5, edgecolor=color_dict[emotion], facecolor='none')
    ax_pca.add_patch(rect_pca)

# Adjust layout
plt.tight_layout()
plt.show()


In [ ]:
# Create an additional subplot to combine all average faces
ax_all = fig.add_subplot(2, len(emotions) + 1, len(emotions) * 2)  # One extra column for comparison

# Combine all average faces into one composite image
all_avg_faces = np.mean([cv2.imread(os.path.join(average_face_dir, f"average_face_{emotion}.png"),
                                    cv2.IMREAD_GRAYSCALE) for emotion in emotions], axis=0)
ax_all.imshow(all_avg_faces, cmap='gray')
ax_all.set_title('All Average Faces')
ax_all.axis('off')

# You can similarly do this for PCA faces if needed
plt.show()